<a href="https://colab.research.google.com/github/jamartinezv2023/deeplearning/blob/main/05_sequence_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Celda 0 — Título y descripción (Markdown)

# 05 - Sequence Models (aplicado al dataset de partidos internacionales)
Este cuaderno explora múltiples arquitecturas de *sequence models* aplicadas al dataset:
"International football results (1872–2017)" (Kaggle).

Secciones:
- 5.0 Cross-validation in time series
- 5.1 Recurrent Neural Networks (SimpleRNN)
- 5.2 LSTM and GRU
- 5.3 Truncated BPTT (demostración)
- 5.4 Text processing (breve, aplicado a columnas text)
- 5.5 Sequence generation (ejemplo didáctico)
- 5.6 Bidirectional RNNs
- 5.7 ELMo (opcional, TF-Hub)
- 5.8 Transformer (basado en MultiHeadAttention)
- 5.9 CNN-LSTM architectures

El notebook prepara los datos, genera secuencias por partido (últimos k partidos por equipo) y entrena modelos comparables.


#Celda 1 — Librerías, montaje Drive y (opcional) Kaggle

In [1]:
# 1. Librerías y montaje de Drive
import os, glob, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import classification_report, confusion_matrix

# Montar Drive (necesario para guardar/leer .npy y modelos)
from google.colab import drive
drive.mount('/content/drive')

# Opcional: instalar kaggle si necesitas descargar directamente el dataset
!pip install -q kaggle


Mounted at /content/drive


#Celda 2 — Descargar dataset desde Kaggle (si no lo tienes)

In [3]:
# DESCARGA DESDE KAGGLE (ejecuta solo si no tienes results.csv en Drive)
# 1) Sube kaggle.json a Colab
# 2) Ejecuta este bloque una vez

# Copiar kaggle.json a la ubicación .kaggle si lo subiste en la raíz de Colab
if os.path.exists("kaggle.json"):
    os.makedirs("/root/.kaggle", exist_ok=True)
    !cp kaggle.json /root/.kaggle/
    !chmod 600 /root/.kaggle/kaggle.json

# Descargar y descomprimir (si no existe ya)
data_zip = "/content/data/international-football-results-from-1872-to-2017.zip"
if not os.path.exists("/content/data/results.csv"):
    !kaggle datasets download -d martj42/international-football-results-from-1872-to-2017 -p /content/data
    !unzip -o /content/data/international-football-results-from-1872-to-2017.zip -d /content/data


Dataset URL: https://www.kaggle.com/datasets/martj42/international-football-results-from-1872-to-2017
License(s): CC0-1.0
  0% 0.00/1.16M [00:00<?, ?B/s]
100% 1.16M/1.16M [00:00<00:00, 561MB/s]
Archive:  /content/data/international-football-results-from-1872-to-2017.zip
  inflating: /content/data/former_names.csv  
  inflating: /content/data/goalscorers.csv  
  inflating: /content/data/results.csv  
  inflating: /content/data/shootouts.csv  


#Celda 3 — Cargar CSV y vista rápida

In [4]:
# Cargar CSV (ajusta la ruta si la tienes en Drive)
csv_paths = [
    "/content/data/results.csv",
    "/content/drive/MyDrive/results.csv",
    "/content/drive/MyDrive/football_project/results.csv"
]
csv_path = None
for p in csv_paths:
    if os.path.exists(p):
        csv_path = p
        break
if csv_path is None:
    raise FileNotFoundError("No se encontró results.csv. Descárgalo o coloca una copia en Drive.")
print("Usando:", csv_path)

df = pd.read_csv(csv_path, parse_dates=['date'])
print(df.shape)
df.head()


Usando: /content/data/results.csv
(48532, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


#Celda 4 — Definir etiqueta y y limpieza breve

In [5]:
# Crear target:
# 0 = HomeWin, 1 = Draw, 2 = AwayWin
def result_label(row):
    if row['home_score'] > row['away_score']:
        return 0
    elif row['home_score'] == row['away_score']:
        return 1
    else:
        return 2

df = df.dropna(subset=['home_score','away_score','date','home_team','away_team']).copy()
df['result'] = df.apply(result_label, axis=1)
df['year'] = df['date'].dt.year

print("Clases (counts):")
print(df['result'].value_counts())


Clases (counts):
result
0    23797
2    13700
1    11035
Name: count, dtype: int64


#Celda 5 — Construir secuencias por partido (ventana k)

In [6]:
# Idea: para cada partido, construimos una secuencia que concatena los últimos k goal_diffs
# de home y de away. Otras features pueden añadirse.
k = 6  # ventana (ajusta si deseas)
# Ordenar por fecha
df = df.sort_values('date').reset_index(drop=True)

# Calculamos goal_diff por partido desde la perspectiva de cada equipo
df['home_goal_diff'] = df['home_score'] - df['away_score']
df['away_goal_diff'] = -df['home_goal_diff']  # perspectiva visitante

# Construir historial por equipo
teams = pd.unique(pd.concat([df['home_team'], df['away_team']]))
history = {t: [] for t in teams}

seqs = []
statics = []
labels = []
indices_kept = []

# Para reproducibilidad: iteramos en orden cronológico
for idx, row in df.iterrows():
    home = row['home_team']
    away = row['away_team']
    date = row['date']
    # last k diffs for home and away
    h_hist = history[home][-k:]
    a_hist = history[away][-k:]
    # pad left with zeros if insuficientes
    h_hist_pad = [0.0]*(k - len(h_hist)) + h_hist
    a_hist_pad = [0.0]*(k - len(a_hist)) + a_hist
    # concatenar -> shape (2*k,)
    seq = np.array(h_hist_pad + a_hist_pad, dtype=np.float32)
    # static features: year (num), neutral (if exists), tournament (encoded later)
    neutral = 0
    if 'neutral' in df.columns:
        neutral = int(bool(row['neutral']))
    # country maybe useful: encode later; for now include year and neutral
    static = np.array([row['year'], neutral], dtype=np.float32)
    seqs.append(seq)
    statics.append(static)
    labels.append(row['result'])
    indices_kept.append(idx)
    # update history: append goal diff to both teams' histories
    history[home].append(row['home_goal_diff'])
    history[away].append(row['away_goal_diff'])

X_seq = np.array(seqs)   # shape (N, 2k)
X_static = np.array(statics)
y = np.array(labels, dtype=np.int32)

# reshape X_seq para modelos que esperan (N, timesteps, channels)
X_seq = X_seq.reshape(X_seq.shape[0], 2*k, 1)

print("Formas resultantes:", X_seq.shape, X_static.shape, y.shape)


Formas resultantes: (48532, 12, 1) (48532, 2) (48532,)


#Celda 6 — Escalado, codificación y split temporal

In [7]:
# Escalar X_static
from sklearn.preprocessing import StandardScaler
scaler_static = StandardScaler()
X_static_scaled = scaler_static.fit_transform(X_static)

# Train/test split por tiempo: usar corte por año para respetar la secuencia temporal
# Ejemplo: entrenar con partidos <= 2010, test > 2010  (ajusta según tamaño)
cut_year = 2010
mask_train = df.loc[indices_kept, 'year'] <= cut_year
# si esto produce sets vacíos, fallback a split aleatorio
if mask_train.sum() < 100:
    Xs_train, Xs_test, Xst_train, Xst_test, y_train, y_test = train_test_split(
        X_seq, X_static_scaled, y, test_size=0.2, random_state=42, stratify=y)
else:
    Xs_train = X_seq[mask_train.values]
    Xs_test  = X_seq[~mask_train.values]
    Xst_train = X_static_scaled[mask_train.values]
    Xst_test  = X_static_scaled[~mask_train.values]
    y_train = y[mask_train.values]
    y_test = y[~mask_train.values]

print("Train/test sizes:", Xs_train.shape[0], Xs_test.shape[0])


Train/test sizes: 34441 14091


#Celda 7 — Guardar arrays a Drive para reproducibilidad

In [8]:
artifacts_dir = "/content/drive/MyDrive/football_project/artifacts"
os.makedirs(artifacts_dir, exist_ok=True)
np.save(os.path.join(artifacts_dir, "X_seq_5k.npy"), X_seq)
np.save(os.path.join(artifacts_dir, "X_static_5k.npy"), X_static_scaled)
np.save(os.path.join(artifacts_dir, "y_5k.npy"), y)
print("Guardado en:", artifacts_dir)


Guardado en: /content/drive/MyDrive/football_project/artifacts


##=== 5.0 Cross-validation in time series (celda explicativa + código) ===

In [9]:
# 5.0 Cross-validation for time series: ejemplo con TimeSeriesSplit
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
# Demostración rápida con el dataset de training (usa indices)
for fold, (train_idx, val_idx) in enumerate(tscv.split(Xs_train)):
    print(f"Fold {fold}: train {len(train_idx)} val {len(val_idx)}")
# Nota: para modelos DL se suele usar validación temporal manual (ej. corte por fecha)


Fold 0: train 5741 val 5740
Fold 1: train 11481 val 5740
Fold 2: train 17221 val 5740
Fold 3: train 22961 val 5740
Fold 4: train 28701 val 5740


##=== 5.1 SimpleRNN (celda modelo + entrenamiento) ===

In [10]:
# 5.1 SimpleRNN baseline
from tensorflow.keras.layers import SimpleRNN

n_classes = len(np.unique(y))
input_seq = keras.Input(shape=Xs_train.shape[1:], name="seq_in")
x = SimpleRNN(64)(input_seq)
x = layers.Dropout(0.3)(x)

input_static = keras.Input(shape=(Xst_train.shape[1],), name="static_in")
s = layers.Dense(32, activation="relu")(input_static)

concat = layers.concatenate([x, s])
out = layers.Dense(64, activation="relu")(concat)
out = layers.Dense(n_classes, activation="softmax")(out)

model_rnn = keras.Model([input_seq, input_static], out)
model_rnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_rnn.summary()

cb = [EarlyStopping(patience=5, restore_best_weights=True),
      ModelCheckpoint("/content/drive/MyDrive/football_project/models/rnn_best.keras", save_best_only=True)]

hist_rnn = model_rnn.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=30, batch_size=128, callbacks=cb)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_in (InputLayer) │ (None, 12, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ simple_rnn          │ (None, 64)        │      4,224 │ seq_in[0][0]      │
│ (SimpleRNN)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ static_in           │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64)        │          0 │ simple_rnn[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │         96 │ static_in[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      6,208 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 3)         │        195 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 10,723 (41.89 KB)

 Trainable params: 10,723 (41.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.4945 - loss: 1.0214 - val_accuracy: 0.5253 - val_loss: 0.9910
Epoch 2/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5360 - loss: 0.9843 - val_accuracy: 0.5291 - val_loss: 0.9893
Epoch 3/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5327 - loss: 0.9822 - val_accuracy: 0.5311 - val_loss: 0.9871
Epoch 4/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5413 - loss: 0.9721 - val_accuracy: 0.5320 - val_loss: 0.9823
Epoch 5/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5384 - loss: 0.9737 - val_accuracy: 0.5256 - val_loss: 0.9863
Epoch 6/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5343 - loss: 0.9785 - val_accuracy: 0.5277 - val_loss: 0.9829
Epoch 7/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5373 - loss: 0.9728 - val_accuracy: 0.5285 - val_loss: 0.9806
Epoch 8/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.5434 - loss: 0.9701 - val_accuracy: 0

##=== 5.2 LSTM y GRU (dos modelos) ===

In [11]:
# 5.2 LSTM
from tensorflow.keras.layers import LSTM, GRU

# LSTM model
seq_in = keras.Input(shape=Xs_train.shape[1:], name="seq")
x = LSTM(64)(seq_in)
x = layers.Dropout(0.3)(x)
stat_in = keras.Input(shape=(Xst_train.shape[1],), name="stat")
s = layers.Dense(32, activation="relu")(stat_in)
c = layers.concatenate([x, s])
h = layers.Dense(64, activation="relu")(c)
out = layers.Dense(n_classes, activation="softmax")(h)
model_lstm = keras.Model([seq_in, stat_in], out)
model_lstm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_lstm.summary()

cb_lstm = [EarlyStopping(patience=5, restore_best_weights=True),
           ModelCheckpoint("/content/drive/MyDrive/football_project/models/lstm_best.keras", save_best_only=True)]
hist_lstm = model_lstm.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=50, batch_size=128, callbacks=cb_lstm)

# GRU model (más ligero)
seq_in2 = keras.Input(shape=Xs_train.shape[1:], name="seq2")
x2 = GRU(64)(seq_in2)
x2 = layers.Dropout(0.3)(x2)
stat_in2 = keras.Input(shape=(Xst_train.shape[1],), name="stat2")
s2 = layers.Dense(32, activation="relu")(stat_in2)
c2 = layers.concatenate([x2, s2])
h2 = layers.Dense(64, activation="relu")(c2)
out2 = layers.Dense(n_classes, activation="softmax")(h2)
model_gru = keras.Model([seq_in2, stat_in2], out2)
model_gru.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_gru.summary()

cb_gru = [EarlyStopping(patience=5, restore_best_weights=True),
          ModelCheckpoint("/content/drive/MyDrive/football_project/models/gru_best.keras", save_best_only=True)]
hist_gru = model_gru.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=50, batch_size=128, callbacks=cb_gru)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq (InputLayer)    │ (None, 12, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     16,896 │ seq[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stat (InputLayer)   │ (None, 2)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │         96 │ stat[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 96)        │          0 │ dropout_1[0][0],  │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      6,208 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 3)         │        195 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,395 (91.39 KB)

 Trainable params: 23,395 (91.39 KB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq2 (InputLayer)   │ (None, 12, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ (None, 64)        │     12,864 │ seq2[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stat2 (InputLayer)  │ (None, 2)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ gru[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32)        │         96 │ stat2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 96)        │          0 │ dropout_2[0][0],  │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64)        │      6,208 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 3)         │        195 │ dense_7[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,363 (75.64 KB)

 Trainable params: 19,363 (75.64 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 10s 34ms/step - accuracy: 0.4924 - loss: 1.0170 - val_accuracy: 0.5207 - val_loss: 0.9872
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5313 - loss: 0.9816 - val_accuracy: 0.5220 - val_loss: 0.9857
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.5362 - loss: 0.9762 - val_accuracy: 0.5227 - val_loss: 0.9814
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.5321 - loss: 0.9808 - val_accuracy: 0.5247 - val_loss: 0.9820
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5355 - loss: 0.9736 - val_accuracy: 0.5252 - val_loss: 0.9820
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.5349 - loss: 0.9755 - val_accuracy: 0.5259 - val_loss: 0.9836
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.5399 - loss: 0.9726 - val_accuracy: 0.5249 - val_loss: 0.9856
Epoch 8/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5405 - loss: 0.9712 - val_acc

##=== 5.3 Truncated BPTT (demostración conceptual) ===

In [12]:
# 5.3 Truncated BPTT: demostración simple.
# En secuencias largas se puede dividir en "segments" y hacer stateful training.
# Aquí haremos una demo conceptual con LSTM stateful usando batch_size fijo.

# ADVERTENCIA: stateful=True requiere manejo cuidadoso de batches y reset_states()
# Este bloque es didáctico y no se recomienda ejecutarlo en producción sin adaptar los datos.

print("Truncated BPTT: revisión teórica y opciones en Keras (stateful LSTM, sequences windows).")


Truncated BPTT: revisión teórica y opciones en Keras (stateful LSTM, sequences windows).


##=== 5.4 Text processing (breve demo) ===

In [13]:
# 5.4 Procesamiento de texto aplicado (ej: torneo o city como texto)
from tensorflow.keras.preprocessing.text import Tokenizer
texts = df['tournament'].fillna("NA").astype(str).values[:2000]  # demo subset
tok = Tokenizer(num_words=5000, oov_token="<OOV>")
tok.fit_on_texts(texts)
seqs_txt = tok.texts_to_sequences(texts)
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded = pad_sequences(seqs_txt, maxlen=10)
print("Ejemplo tokenizado y padded shape:", padded.shape)


Ejemplo tokenizado y padded shape: (2000, 10)


##=== 5.5 Sequence generation (ejemplo simple de char-RNN) ===

In [14]:
# 5.5 Generación de secuencias - ejemplo didáctico (caracteres)
# Dataset pequeño de demostración
sample_text = "football machine learning example"
chars = sorted(list(set(sample_text)))
char2idx = {c:i for i,c in enumerate(chars)}
idx2char = {i:c for c,i in char2idx.items()}

# preparar dataset (char to char)
seq_len = 10
Xg, yg = [], []
for i in range(len(sample_text)-seq_len):
    s = sample_text[i:i+seq_len]
    t = sample_text[i+seq_len]
    Xg.append([char2idx[c] for c in s])
    yg.append(char2idx[t])
Xg = np.array(Xg); yg = np.array(yg)
# pequeño modelo char-RNN
model_char = keras.Sequential([
    layers.Embedding(input_dim=len(chars), output_dim=8, input_length=seq_len),
    layers.LSTM(64),
    layers.Dense(len(chars), activation='softmax')
])
model_char.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model_char.fit(Xg, yg, epochs=50, verbose=0)
print("Char-RNN entrenado (demo).")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Char-RNN entrenado (demo).


##=== 5.6 Bidirectional RNNs ===

In [15]:
# 5.6 Bidirectional LSTM
from tensorflow.keras.layers import Bidirectional
seq_in = keras.Input(shape=Xs_train.shape[1:], name="seq_bi")
x = Bidirectional(LSTM(64))(seq_in)
x = layers.Dropout(0.3)(x)
stat_in = keras.Input(shape=(Xst_train.shape[1],), name="stat_bi")
s = layers.Dense(32, activation='relu')(stat_in)
c = layers.concatenate([x, s])
o = layers.Dense(n_classes, activation='softmax')(layers.Dense(64, activation='relu')(c))

model_bi = keras.Model([seq_in, stat_in], o)
model_bi.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_bi.summary()

cb_bi = [EarlyStopping(patience=5, restore_best_weights=True),
         ModelCheckpoint("/content/drive/MyDrive/football_project/models/bi_lstm_best.keras", save_best_only=True)]
hist_bi = model_bi.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=30, batch_size=128, callbacks=cb_bi)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ seq_bi (InputLayer) │ (None, 12, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 128)       │     33,792 │ seq_bi[0][0]      │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stat_bi             │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 128)       │          0 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 32)        │         96 │ stat_bi[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 160)       │          0 │ dropout_3[0][0],  │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 64)        │     10,304 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 3)         │        195 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 44,387 (173.39 KB)

 Trainable params: 44,387 (173.39 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 12s 39ms/step - accuracy: 0.5093 - loss: 1.0017 - val_accuracy: 0.5205 - val_loss: 0.9879
Epoch 2/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.5297 - loss: 0.9798 - val_accuracy: 0.5191 - val_loss: 0.9866
Epoch 3/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.5285 - loss: 0.9808 - val_accuracy: 0.5227 - val_loss: 0.9855
Epoch 4/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.5358 - loss: 0.9746 - val_accuracy: 0.5245 - val_loss: 0.9824
Epoch 5/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - accuracy: 0.5363 - loss: 0.9766 - val_accuracy: 0.5224 - val_loss: 0.9835
Epoch 6/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.5367 - loss: 0.9742 - val_accuracy: 0.5217 - val_loss: 0.9849
Epoch 7/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.5383 - loss: 0.9703 - val_accuracy: 0.5213 - val_loss: 0.9853
Epoch 8/30
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - accuracy: 0.5374 - loss: 0.9707 - val_acc

##=== 5.7 ELMo (opcional) ===

In [16]:
# 5.7 ELMo (opcional) - requiere internet y tensorflow_hub
# NOTA: TF-Hub y ELMo pueden requerir versiones específicas de TF y mucho tiempo/espacio.
print("ELMo es opcional. Para usarlo, instala tensorflow-hub y carga el modelo contextual.")
# Ejemplo (no ejecutarlo si no tienes recursos):
# !pip install -q tensorflow-hub
# import tensorflow_hub as hub
# elmo = hub.load("https://tfhub.dev/google/elmo/3")
# emb = elmo.signatures['default'](tf.constant(["This is a test"]))['elmo']


ELMo es opcional. Para usarlo, instala tensorflow-hub y carga el modelo contextual.


##=== 5.8 Transformer (mini-implementación) ===

In [20]:
# ================================
# Imports necesarios
# ================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    LayerNormalization,
    MultiHeadAttention,
    Add,
    GlobalAveragePooling1D,
    Input
)


In [21]:
# 5.8 Transformer-like small model usando MultiHeadAttention
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add

def transformer_block(x, head_size=32, num_heads=2, ff_dim=64, dropout=0.1):
    attn_out = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    attn_out = Dropout(dropout)(attn_out)
    out1 = LayerNormalization(epsilon=1e-6)(Add()([x, attn_out]))
    ff = layers.Dense(ff_dim, activation="relu")(out1)
    ff = layers.Dense(x.shape[-1])(ff)
    ff = Dropout(dropout)(ff)
    return LayerNormalization(epsilon=1e-6)(Add()([out1, ff]))

# Preparar input: transformer expects shape (batch, timesteps, features)
# Xs_train currently (batch, timesteps, 1) - good
seq_input = keras.Input(shape=Xs_train.shape[1:], name="tr_seq")  # (timesteps, 1)
x = layers.Dense(32)(seq_input)  # project to d_model=32
x = transformer_block(x, head_size=16, num_heads=2, ff_dim=64)
x = layers.GlobalAveragePooling1D()(x)

static_in = keras.Input(shape=(Xst_train.shape[1],), name="tr_stat")
s = layers.Dense(32, activation="relu")(static_in)
concat = layers.concatenate([x, s])
h = layers.Dense(64, activation="relu")(concat)
out = layers.Dense(n_classes, activation="softmax")(h)
model_tr = keras.Model([seq_input, static_in], out)
model_tr.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_tr.summary()

cb_tr = [EarlyStopping(patience=5, restore_best_weights=True),
         ModelCheckpoint("/content/drive/MyDrive/football_project/models/transformer_best.keras", save_best_only=True)]
hist_tr = model_tr.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=50, batch_size=128, callbacks=cb_tr)


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ tr_seq (InputLayer) │ (None, 12, 1)     │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 12, 32)    │         64 │ tr_seq[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 12, 32)    │      4,224 │ dense_17[0][0],   │
│ (MultiHeadAttentio… │                   │            │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_7 (Dropout) │ (None, 12, 32)    │          0 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 12, 32)    │          0 │ dense_17[0][0],   │
│                     │                   │            │ dropout_7[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 12, 32)    │         64 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_18 (Dense)    │ (None, 12, 64)    │      2,112 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_19 (Dense)    │ (None, 12, 32)    │      2,080 │ dense_18[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 12, 32)    │          0 │ dense_19[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 12, 32)    │          0 │ layer_normalizat… │
│                     │                   │            │ dropout_8[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 12, 32)    │         64 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tr_stat             │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_20 (Dense)    │ (None, 32)        │         96 │ tr_stat[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 64)        │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ dense_20[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_21 (Dense)    │ (None, 64)        │      4,160 │ concatenate_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_22 (Dense)    │ (None, 3)         │        195 │ dense_21[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,059 (51.01 KB)

 Trainable params: 13,059 (51.01 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.4891 - loss: 1.0445 - val_accuracy: 0.4789 - val_loss: 1.0470
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.4996 - loss: 1.0324 - val_accuracy: 0.4789 - val_loss: 1.0474
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - accuracy: 0.4996 - loss: 1.0303 - val_accuracy: 0.4673 - val_loss: 1.0500
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step - accuracy: 0.4973 - loss: 1.0324 - val_accuracy: 0.4790 - val_loss: 1.0468
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.4962 - loss: 1.0324 - val_accuracy: 0.4805 - val_loss: 1.0437
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.4933 - loss: 1.0332 - val_accuracy: 0.4750 - val_loss: 1.0466
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4963 - loss: 1.0305 - val_accuracy: 0.4784 - val_loss: 1.0440
Epoch 8/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.4967 - loss: 1.0308 - val_accu

##=== 5.9 CNN-LSTM (arquitectura combinada) ===

In [22]:
# 5.9 CNN-LSTM: Conv1D -> MaxPool -> LSTM -> concat static
seq_in = keras.Input(shape=Xs_train.shape[1:], name="cnn_lstm_seq")
c = layers.Conv1D(32, 3, activation="relu", padding="same")(seq_in)
c = layers.MaxPooling1D(2)(c)
c = layers.Conv1D(64, 3, activation="relu", padding="same")(c)
c = layers.LSTM(64)(c)
c = layers.Dropout(0.3)(c)

stat_in = keras.Input(shape=(Xst_train.shape[1],), name="cnn_lstm_stat")
s = layers.Dense(32, activation="relu")(stat_in)
m = layers.concatenate([c, s])
h = layers.Dense(64, activation="relu")(m)
out = layers.Dense(n_classes, activation="softmax")(h)

model_cnn_lstm = keras.Model([seq_in, stat_in], out)
model_cnn_lstm.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model_cnn_lstm.summary()

cb_cnnlstm = [EarlyStopping(patience=5, restore_best_weights=True),
              ModelCheckpoint("/content/drive/MyDrive/football_project/models/cnn_lstm_best.keras", save_best_only=True)]
hist_cnnlstm = model_cnn_lstm.fit([Xs_train, Xst_train], y_train, validation_split=0.2, epochs=50, batch_size=128, callbacks=cb_cnnlstm)


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cnn_lstm_seq        │ (None, 12, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 12, 32)    │        128 │ cnn_lstm_seq[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_1     │ (None, 6, 32)     │          0 │ conv1d_2[0][0]    │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 6, 64)     │      6,208 │ max_pooling1d_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ (None, 64)        │     33,024 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cnn_lstm_stat       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 64)        │          0 │ lstm_4[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 32)        │         96 │ cnn_lstm_stat[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_6       │ (None, 96)        │          0 │ dropout_9[0][0],  │
│ (Concatenate)       │                   │            │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      6,208 │ concatenate_6[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_25 (Dense)    │ (None, 3)         │        195 │ dense_24[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45,859 (179.14 KB)

 Trainable params: 45,859 (179.14 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.5068 - loss: 1.0141 - val_accuracy: 0.5191 - val_loss: 0.9893
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5373 - loss: 0.9771 - val_accuracy: 0.5210 - val_loss: 0.9880
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5356 - loss: 0.9743 - val_accuracy: 0.5159 - val_loss: 1.0014
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.5382 - loss: 0.9705 - val_accuracy: 0.5245 - val_loss: 0.9869
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.5411 - loss: 0.9679 - val_accuracy: 0.5256 - val_loss: 0.9814
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5402 - loss: 0.9670 - val_accuracy: 0.5266 - val_loss: 0.9930
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5410 - loss: 0.9701 - val_accuracy: 0.5300 - val_loss: 0.9825
Epoch 8/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.5375 - loss: 0.9724 - val_accu

##Celda final — Comparativa rápida de resultados (imprime accuracy de test donde se haya guardado)

In [23]:
# Cargar modelos guardados (si existen) y evaluar en test
models_to_check = {
    "RNN": "/content/drive/MyDrive/football_project/models/rnn_best.keras",
    "LSTM": "/content/drive/MyDrive/football_project/models/lstm_best.keras",
    "GRU": "/content/drive/MyDrive/football_project/models/gru_best.keras",
    "BiLSTM": "/content/drive/MyDrive/football_project/models/bi_lstm_best.keras",
    "TRANSFORMER": "/content/drive/MyDrive/football_project/models/transformer_best.keras",
    "CNN-LSTM": "/content/drive/MyDrive/football_project/models/cnn_lstm_best.keras"
}

for name, path in models_to_check.items():
    if os.path.exists(path):
        m = keras.models.load_model(path)
        loss, acc = m.evaluate([Xs_test, Xst_test], y_test, verbose=0)
        print(f"{name}: Test acc = {acc:.4f}")
    else:
        print(f"{name}: no encontrado en {path}")


RNN: Test acc = 0.5315
LSTM: Test acc = 0.5265
GRU: Test acc = 0.5298
BiLSTM: Test acc = 0.5280
TRANSFORMER: Test acc = 0.4738
CNN-LSTM: Test acc = 0.5315
